# Small Experiment Data Analysis

In [1]:
import json
from six.moves import cPickle
import os
import statistics

from IPython.core.display import HTML 
from IPython.core.display import Image, display

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
"""
Load Coco data
"""
COCO_PATH = 'data/coco/'
COCO_ANNO_PATH = COCO_PATH + 'annotations/'

# load the data from captions
with open(COCO_ANNO_PATH + 'captions_train2014.json') as f:
    coco_captions = json.load(f)
with open(COCO_ANNO_PATH + 'captions_train2014.json') as f:
    coco_captions = json.load(f)

    
# build the reverse dictionary, from img_id to captions, img_infos, and annotations
img_captions = {}
for img_info in coco_captions['images']:
    mid = int(img_info['id'])
    if not mid in img_captions:
        img_captions[mid] = {}
    img_captions[mid]['image'] = img_info

for cap_info in coco_captions['annotations']:
    mid = int(cap_info['image_id'])
    if not 'annotation' in img_captions[mid]:
        img_captions[mid]['annotation'] = []
        img_captions[mid]['captions'] = []
    img_captions[mid]['annotation'].append(cap_info)


In [4]:
IN_FILE_NAME  = 'high_coexist_img_ids.pkl'
OUT_FILE_NAME = 'highexist_gaussian'
IN_DIR        = 'input'
OUT_DIR       = 'output'

def get_stat(data):
    if len(data)==0: return;
    print ""
    print "Total:\t%d;\t\tMean:\t\t%f;\tNonezero:\t%d"%(len(data), statistics.mean(data), len(filter(lambda x:x!=0, data)))
    print "Median:\t%f;\tMedian(H):\t%f;\tMedian(L):\t%f;"%(statistics.median(data), statistics.median_high(data), statistics.median_low(data))
    print "Max:\t%f;\tMin:\t\t%f;\tStd:\t\t%f"%(max(data), min(data),statistics.stdev(data))
    print "\n"
    
def look_up_image(title, idx, in_ids, vis, scores):
    print "[%d]%s\n"%(idx,title)
    print "\tScore:%s;\tImageId:%s\n"%(scores[idx], in_ids[idx])
    print "\tOriginal Caption:\n\t\t%s;\n\tAblation Caption:\n\t\t%s;"%(vis[idx*2+1]['caption'],vis[idx*2]['caption'])
    print "\tAnnotated Captions:\n"
    img_id = in_ids[idx]
    for idx, note in enumerate(img_captions[img_id]['annotation']):
        print "\t\t%d. %s\n"%(idx+1, note['caption'].trim())
    display(Image(url= img_captions[img_id]['image']['coco_url']))

def get_expr_summary(in_file=IN_FILE_NAME, out_file=OUT_FILE_NAME, in_path=IN_DIR, out_path=OUT_DIR, num=5):
    in_ids  = cPickle.load(open(os.path.join(in_path, in_file)))
    scores  = cPickle.load(open(os.path.join(out_path, "scores_%s.pkl"%out_file)))
    vis     = json.load(open(os.path.join(out_path, 'vis_%s.json'%out_file)))

    print "Statistics for scores of gaussian filtered ablations:"
    get_stat(scores)

    scores_idx = zip(range(len(scores)), scores)
    sorted_scores_idx = sorted(scores_idx, key=lambda x: x[1])   # sort by score
    for idx, _ in sorted_scores_idx[:num]:
        look_up_image("Zero Image",idx, in_ids, vis, scores)

get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)

Statistics for scores of gaussian filtered ablations:

Total:	2026;		Mean:		0.840647;	Nonezero:	2016
Median:	1.000000;	Median(H):	1.000000;	Median(L):	1.000000;
Max:	1.000000;	Min:		0.000000;	Std:		0.277668


[108]Zero Image

	Score:0.0;	ImageId:524984

	Original Caption:
		a stop sign is on the corner of a street;
	Ablation Caption:
		a person holding a glass of wine in a room;
	Annotated Captions:



KeyError: '524984'

In [49]:
blkin     = cPickle.load(open(os.path.join(IN_DIR, B_OUT_FILE)))
blkvis    = json.load(open(os.path.join(B_TMP_DIR, 'vis.json')))
glkscores = cPickle.load(open(os.path.join(OUT_DIR, B_OUT_FILE)))

ValueError: No JSON object could be decoded

In [ ]:
os.listdir()